In [1]:
import itertools

# This is the function you need to implement
def exhaustive(tokens, distributions, transitions, labels):
    # TODO
    combinations = itertools.product(labels, repeat=len(tokens))
    scores = [1] * len(labels) ** len(tokens)
    combos = []

    # To iterate:
    for i, combo in enumerate(combinations):
        combos.append(combo)
        for j, char in enumerate(combo):
            scores[i] *= distributions[j][char]
        combo_exp = ('START',) + combo + ('END',)
        for j in range(len(combo_exp) - 1):
            scores[i] *= transitions[(combo_exp[j], combo_exp[j+1])]

    max_val = max(scores)
    max_indices = [i for i, val in enumerate(scores) if val == max_val]
    max_combos = [combos[i] for i in max_indices]
    max_combos.sort(reverse=True)

    return max_val, max_combos[0]

# Sample data test
tokens = ["Sydney", "is", "nice"]
distributions = [
    {"LOC": 0.9, "O": 0.1},
    {"LOC": 0.05, "O": 0.95},
    {"LOC": 0.05, "O": 0.95},
]
transitions = {
    ("START", "O"): 0.8,
    ("START", "LOC"): 0.2,
    ("START", "END"): 0.0,
    ("O", "END"): 0.05,
    ("O", "O"): 0.9,
    ("O", "LOC"): 0.05,
    ("LOC", "END"): 0.05,
    ("LOC", "O"): 0.8,
    ("LOC", "LOC"): 0.2,
}
labels = {"LOC", "O"}

answer = exhaustive(tokens, distributions, transitions, labels)
if abs(answer[0] - 0.0058482000000000004) > 1e-10:
    print("Error in score")
if ' '.join(answer[1]) != 'LOC O O':
    print("Error in sequence")

In [2]:
# This is the function you need to implement
def beam(tokens, distributions, transitions, labels, k):
    # TODO
    beam = [{'seq': ('START',), 'prob': 1}]

    for i in range(len(tokens)):
        new_beam = []
        for seq in beam:
            for label in labels:
                new_beam.append({
                    'seq': seq['seq'] + (label,), 
                    'prob': seq['prob'] * distributions[i][label] * transitions[(seq['seq'][-1], label)]
                })
        new_beam = sorted(new_beam, key=lambda x: x['prob'], reverse=True)
        beam = new_beam[:k]

    new_beam = []
    for seq in beam:
        label = 'END'
        new_beam.append({
            'seq': seq['seq'] + (label,), 
            'prob': seq['prob'] * transitions[(seq['seq'][-1], label)]
        })
    new_beam = sorted(new_beam, key=lambda x: x['prob'], reverse=True)
    beam = new_beam[:k]

    return beam[0]['prob'], beam[0]['seq'][1:-1]

# Sample data test
tokens = ["Sydney", "is", "nice"]
distributions = [
    {"LOC": 0.9, "O": 0.1},
    {"LOC": 0.05, "O": 0.95},
    {"LOC": 0.05, "O": 0.95},
]
transitions = {
    ("START", "O"): 0.8,
    ("START", "LOC"): 0.2,
    ("START", "END"): 0.0,
    ("O", "END"): 0.05,
    ("O", "O"): 0.9,
    ("O", "LOC"): 0.05,
    ("LOC", "END"): 0.05,
    ("LOC", "O"): 0.8,
    ("LOC", "LOC"): 0.2,
}
labels = {"LOC", "O"}
k = 5

answer = beam(tokens, distributions, transitions, labels, k)
print(answer)
if abs(answer[0] - 0.0058482000000000004) > 1e-10:
    print("Error in score")
if ' '.join(answer[1]) != 'LOC O O':
    print("Error in sequence")

(0.0058482000000000004, ('LOC', 'O', 'O'))


In [81]:
# This is the function you need to implement
def viterbi(tokens, distributions, transitions, labels):
    # TODO
    storage = [[{'prob': 1, 'prev': 0}]]
    
    for i in range(len(tokens) + 1):
        step = []
        options = list(labels) if i > 0 else ['START']
        labels_plus = labels if i < len(tokens) else {'END'}
        for label in labels_plus:
            probs = []
            for j, prev in enumerate(storage[i]):
                prob = transitions[(options[j], label)] * prev['prob']
                probs.append({'idx': j, 'label': options[j], 'prob': prob})
            
            max_val = max([d['prob'] for d in probs])
            max_probs = [d for d in probs if d['prob'] == max_val]
            max_probs = sorted(max_probs, key=lambda x: x['label'], reverse=True)

            dist = distributions[i][label] if i < len(tokens) else 1
            step.append({'prob': dist * max_probs[0]['prob'], 'prev': max_probs[0]['idx']})
        storage.append(step)

    seq = []
    best = 0
    for k in storage[::-1]:
        seq.append(list(labels)[best])
        best = k[best]['prev']

    return storage[-1][0]['prob'], tuple(seq[1:-1][::-1])

# Sample data test
tokens = ["Sydney", "is", "nice"]
distributions = [
    {"LOC": 0.9, "O": 0.1},
    {"LOC": 0.05, "O": 0.95},
    {"LOC": 0.05, "O": 0.95},
]
transitions = {
    ("START", "O"): 0.8,
    ("START", "LOC"): 0.2,
    ("START", "END"): 0.0,
    ("O", "END"): 0.05,
    ("O", "O"): 0.9,
    ("O", "LOC"): 0.05,
    ("LOC", "END"): 0.05,
    ("LOC", "O"): 0.8,
    ("LOC", "LOC"): 0.2,
}
labels = {"LOC", "O"}

answer = viterbi(tokens, distributions, transitions, labels)
print(answer)
if abs(answer[0] - 0.0058482000000000004) > 1e-10:
    print("Error in score")
if ' '.join(answer[1]) != 'LOC O O':
    print("Error in sequence")

(0.0058482000000000004, ('LOC', 'O', 'O'))
